## 1) Fase di importazione delle librerie

In questa sezione iniziale vengono importate tutte le librerie Python necessarie per l'implementazione dell'algoritmo di generazione dell'orario scolastico. Le librerie utilizzate includono `pandas` e `numpy` per la gestione e manipolazione efficiente dei dati tabellari, `random` per la generazione di scelte casuali durante le operazioni euristiche, e `copy` per creare copie profonde delle strutture dati senza modificare gli originali. Il modulo `math` fornisce funzioni matematiche di base, mentre `time` permette di tracciare i tempi di esecuzione. Infine, `collections` offre strutture dati specializzate come `defaultdict` per dizionari con valori predefiniti e `Counter` per conteggi efficienti, che risultano particolarmente utili nella gestione delle assegnazioni e nel calcolo delle statistiche.

In [1]:
import pandas as pd
import numpy as np
import random
import copy
import math
import time
from collections import defaultdict, Counter

## 2) Definizione del problema

Questo blocco definisce i parametri fondamentali del problema di scheduling scolastico. Vengono stabiliti i giorni della settimana lavorativi attraverso la lista `GIORNI`, che comprende sei giorni da lunedì a sabato. Gli slot orari giornalieri sono rappresentati dalla lista `SLOTS` con valori da 1 a 5, corrispondenti alle cinque ore di lezione quotidiane. Il parametro `MAX_ORE_GIORNO` fissa a 4 il numero massimo di ore che un professore può insegnare in un singolo giorno, garantendo un carico di lavoro sostenibile. La variabile `MAX_ORE_BUCHE` limita a 1 il numero massimo di ore libere che un professore può avere tra le lezioni nello stesso giorno, minimizzando i tempi morti. Infine, la lista `CLASSI` specifica le sei classi per cui generare l'orario: 3A, 3B, 4A, 4C, 5B e 5C. Questi vincoli costituiscono la base su cui verrà costruita la soluzione del problema.

In [2]:
# Definizione dei set del problema
GIORNI = ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato"]
SLOTS = [1, 2, 3, 4, 5]  # 5 slot giornalieri
MAX_ORE_GIORNO = 4  # Numero massimo di ore giornaliere per un professore
MAX_ORE_BUCHE = 1  # Numero massimo di ore buche per un professore in un giorno

# Lista delle classi (utilizzando l'elenco fornito)
CLASSI = ["3A", "3B", "4A", "4C", "5B", "5C"]


## 3) Funzione per la lettura delle cattedre ed elenco cattedre

Questa sezione implementa il parsing dei dati relativi alle cattedre dei professori. La funzione `leggi_cattedre()` prende in input una stringa formattata che descrive le assegnazioni di ciascun professore e la trasforma in una struttura dati dizionario più facilmente manipolabile. Per ogni riga del formato "Professore: X ore in Classe, Y ore in Classe", la funzione estrae il nome del docente e le sue assegnazioni, creando per ciascuno una lista di tuple contenenti la classe e il numero di ore da insegnare. La variabile `dati_cattedre` contiene l'esempio concreto con 34 professori e le loro rispettive assegnazioni alle sei classi, rappresentando il dataset completo su cui l'algoritmo dovrà operare. Questo approccio strutturato permette di gestire agevolmente le 180 ore totali di lezione da distribuire, facilitando successivi controlli e verifiche sulla completezza dell'orario generato.

In [3]:
# Funzione per leggere le cattedre dei professori da un file o una stringa
def leggi_cattedre(dati_cattedre):
    cattedre = {}

    # Parsing dei dati delle cattedre
    lines = dati_cattedre.strip().split('\n')
    for line in lines:
        if not line.strip() or ':' not in line:
            continue

        parts = line.split(':')
        professore = parts[0].strip()
        assignments = parts[1].strip()

        cattedre[professore] = []

        # Estrazione delle assegnazioni per il professore
        for assignment in assignments.split(','):
            assignment = assignment.strip()
            if not assignment:
                continue

            parts = assignment.split('in')
            if len(parts) != 2:
                continue

            ore = int(parts[0].strip().split()[0])
            classe = parts[1].strip()

            cattedre[professore].append((classe, ore))

    return cattedre

# Esempio di dati delle cattedre
dati_cattedre = """
Smith: 3 ore in 3B, 3 ore in 5C
Johnson: 4 ore in 3A
Williams: 2 ore in 3A
Brown: 3 ore in 4C
Jones: 7 ore in 4C
Miller: 2 ore in 5B
Davis: 2 ore in 5C
Garcia: 3 ore in 4A
Martinez: 4 ore in 4C
Taylor: 2 ore in 3B, 2 ore in 5B
Anderson: 2 ore in 4C
Thomas: 3 ore in 3A
Jackson: 2 ore in 4A
White: 2 ore in 5B, 2 ore in 4C
Harris: 4 ore in 3A
Martin: 2 ore in 3B, 2 ore in 4A, 2 ore in 5C
Thompson: 2 ore in 4C
Moore: 2 ore in 3A
Allen: 6 ore in 4C
Young: 7 ore in 3B, 7 ore in 4A
King: 1 ore in 3A, 1 ore in 5B
Wright: 7 ore in 5C
Lopez: 4 ore in 4A, 4 ore in 3B
Hill: 4 ore in 5C
Scott: 2 ore in 3A
Green: 2 ore in 3B, 2 ore in 5B, 2 ore in 4A, 2 ore in 5C
Adams: 4 ore in 4A
Baker: 6 ore in 4A
Gonzalez: 5 ore in 3A, 6 ore in 5C
Nelson: 6 ore in 3B, 6 ore in 5B
Carter: 4 ore in 4C, 4 ore in 5C
Mitchell: 7 ore in 3A, 7 ore in 5B
Perez: 4 ore in 5B
Roberts: 4 ore in 3B, 4 ore in 5B
"""


## 4) Funzione generatrice di giorni liberi

Questo blocco definisce la funzione `genera_preferenze_giorni_liberi()` che crea le preferenze di ciascun professore riguardo ai giorni liberi. Il sistema implementa un meccanismo di pesi differenziati: la prima scelta del professore riceve peso 2, la seconda scelta peso 1, mentre i rimanenti giorni hanno peso 0. Questa struttura consente all'algoritmo di ottimizzazione di tenere conto delle preferenze individuali durante la costruzione dell'orario, cercando di massimizzare la soddisfazione complessiva. La funzione è progettata per gestire sia preferenze specifiche predefinite che generazione casuale per professori non presenti nell'esempio, garantendo flessibilità nell'utilizzo. L'output è un dizionario che associa a ogni professore un altro dizionario contenente, per ciascun giorno della settimana, il peso corrispondente alla sua preferenza.

In [4]:
def genera_preferenze_giorni_liberi(professori, giorni):
    """
    Genera preferenze per i giorni liberi:
    - Prima scelta: peso 2
    - Seconda scelta: peso 1
    - Altre scelte: peso 0
    """
    preferenze = {}

    # Esempio specifico di preferenze (generato manualmente)
    esempio_preferenze = {}

    # Per ogni professore, prendi le preferenze dall'esempio o genera casualmente se non presenti
    for prof in professori:
        if prof in esempio_preferenze:
            preferenze[prof] = esempio_preferenze[prof]
        else:
            # Per eventuali professori non presenti nell'esempio, genera casualmente
            prima_scelta, seconda_scelta = random.sample(giorni, 2)
            prof_pref = {g: 0 for g in giorni}
            prof_pref[prima_scelta] = 2   # Prima scelta (peso 2)
            prof_pref[seconda_scelta] = 1  # Seconda scelta (peso 1)
            preferenze[prof] = prof_pref

    return preferenze

## 5) Funzione per il calcolo del punteggio di un assegnamento

In questa sezione vengono implementate le funzioni fondamentali per valutare la qualità di un possibile assegnamento di lezione. La funzione principale `calcola_punteggio()` effettua una serie di controlli progressivi: inizialmente verifica i vincoli hard (inviolabili) come la compatibilità professore-classe, il limite di ore giornaliere, l'assenza di sovrapposizioni e il massimo di 2 lezioni per classe al giorno. Se uno di questi vincoli è violato, restituisce un punteggio negativo infinito per scartare immediatamente l'assegnamento. Successivamente, attraverso le funzioni ausiliarie `ha_ore_buche()` e `verifica_sequenza_lezioni()`, controlla che le ore buche non superino il limite consentito e che le lezioni della stessa materia non siano distanziate di 2 o più slot. Per gli assegnamenti ammissibili, il punteggio viene calcolato combinando diversi fattori: un bonus base, premi per l'accorpamento di lezioni consecutive della stessa materia (peso +50), penalità per l'uso di giorni preferiti come liberi, bonus per professori con minore flessibilità, e penalità per squilibri nella distribuzione delle ore. Questo approccio euristico guida l'algoritmo greedy verso soluzioni di qualità elevata.

In [5]:
def calcola_punteggio(prof, classe, giorno, slot, orario_attuale, cattedre_rimanenti, preferenze_giorni):
    punteggio = 0

    # Verifica se il professore può insegnare in questa classe
    if classe not in [c for c, _ in cattedre_rimanenti.get(prof, [])]:
        return float('-inf')

    # Verifica ore massime giornaliere
    ore_prof_giorno = sum(1 for g, s, p, c in orario_attuale if p == prof and g == giorno)
    if ore_prof_giorno >= MAX_ORE_GIORNO:
        return float('-inf')

    # Verifica sovrapposizioni
    for g, s, p, c in orario_attuale:
        if g == giorno and s == slot:
            if p == prof or c == classe:
                return float('-inf')

    # Verifica massimo 2 lezioni per classe al giorno
    lezioni_stessa_classe = sum(1 for g, s, p, c in orario_attuale
                             if g == giorno and p == prof and c == classe)
    if lezioni_stessa_classe >= 2:
        return float('-inf')

    # Verifica ore buche
    slots_occupati = [s for g, s, p, c in orario_attuale if g == giorno and p == prof]
    slots_occupati.append(slot)
    slots_occupati.sort()

    ore_buche = ha_ore_buche(slots_occupati)
    if ore_buche > MAX_ORE_BUCHE:
        return float('-inf')

    # Verifica sequenza lezioni (non distanziate di 2+ slot)
    if not verifica_sequenza_lezioni(prof, classe, giorno, slot, orario_attuale):
        return float('-inf')

    # Punteggio base positivo per incoraggiare assegnazioni
    punteggio += 10

    # Premia l'accorpamento (due lezioni consecutive della stessa materia)
    if lezioni_stessa_classe == 1:
        # Verifica se le lezioni sarebbero consecutive
        altre_lezioni = [s for g, s, p, c in orario_attuale
                       if g == giorno and p == prof and c == classe]
        if len(altre_lezioni) == 1 and abs(altre_lezioni[0] - slot) == 1:
            punteggio += 50

    # Considera le preferenze per i giorni liberi
    # Penalizza l'assegnazione nei giorni più preferiti
    punteggio -= preferenze_giorni.get(prof, {}).get(giorno, 0) * 10

    # Premia professori con poca flessibilità
    num_classi_prof = len(set(c for c, _ in cattedre_rimanenti.get(prof, [])))
    punteggio += (6 - num_classi_prof) * 5

    # Distribuisci uniformemente le ore
    ore_per_giorno = {}
    for g in GIORNI:
        ore_per_giorno[g] = sum(1 for gg, _, pp, _ in orario_attuale if pp == prof and gg == g)

    # Penalizza l'assegnazione in giorni già carichi
    punteggio -= ore_per_giorno.get(giorno, 0) * 5

    return punteggio
def ha_ore_buche(slots_occupati):
    """
    Verifica se ci sono ore buche nei slot occupati.
    Una ora buca è quando c'è un gap tra due slot occupati.
    """
    if len(slots_occupati) <= 1:
        return 0

    slots_occupati.sort()
    ore_buche = 0

    for i in range(len(slots_occupati) - 1):
        gap = slots_occupati[i + 1] - slots_occupati[i] - 1
        ore_buche += max(0, gap)  # Conta solo i gap positivi

    return ore_buche

def verifica_sequenza_lezioni(prof, classe, giorno, slot, orario_attuale):
    """
    Verifica che un professore non insegni nella stessa classe con
    lezioni distanziate di 2 slot o più
    """
    slots_occupati_classe = [s for g, s, p, c in orario_attuale
                           if g == giorno and p == prof and c == classe]

    if not slots_occupati_classe:
        return True

    # Aggiungi lo slot attuale
    slots_futuri = slots_occupati_classe + [slot]
    slots_futuri.sort()

    # Verifica se ci sono lezioni distanziate da 2 o più slot
    for i in range(len(slots_futuri) - 1):
        if slots_futuri[i+1] - slots_futuri[i] >= 3:  # distanza di 2 o più slot
            return False

    return True

## 6) Funzione principale dell'algoritmo greedy

Questo blocco contiene il cuore dell'algoritmo greedy per la generazione iniziale dell'orario. La funzione `orario_scolastico()` opera iterativamente selezionando a ogni passo l'assegnamento migliore tra tutte le possibili combinazioni di professore, classe, giorno e slot. L'algoritmo mantiene traccia delle ore rimanenti da assegnare per ogni cattedra, degli slot già occupati e della distribuzione oraria di ciascun professore nei vari giorni. A ogni iterazione, valuta sistematicamente tutte le opzioni disponibili utilizzando la funzione `calcola_punteggio()` e sceglie quella con il punteggio massimo, garantendo così una costruzione incrementale dell'orario che rispetta i vincoli e ottimizza gli obiettivi. Durante l'esecuzione, l'algoritmo aggiorna dinamicamente le strutture dati e, quando un professore raggiunge il limite di giorni lavorativi, gli assegna automaticamente un giorno libero privilegiando le sue preferenze. Il processo continua finché tutte le ore vengono assegnate o non è più possibile trovare assegnamenti validi. Al termine, fornisce un report dettagliato indicando il numero di ore assegnate e, eventualmente, le ore rimaste non allocate con i relativi dettagli per professore e classe.

In [6]:
def orario_scolastico(cattedre, classi, giorni, slots, preferenze_giorni):
    # Inizializza l'orario vuoto
    orario = []

    # Copia delle cattedre rimanenti
    cattedre_rimanenti = {}
    for prof, assegnazioni in cattedre.items():
        cattedre_rimanenti[prof] = assegnazioni.copy()

    # Conteggio ore totali da assegnare
    ore_totali = sum(ore for prof in cattedre_rimanenti for _, ore in cattedre_rimanenti[prof])
    ore_iniziali = ore_totali
    print(f"Ore totali da assegnare: {ore_totali}")

    # Tracciamento slot già assegnati
    slot_assegnati = set()

    # Tracciamento delle ore in cui ogni prof insegna in ogni giorno
    ore_prof_giorno = {prof: {giorno: 0 for giorno in giorni} for prof in cattedre.keys()}

    # Tracciamento giorni liberi
    giorni_liberi_assegnati = {}

    # Finché ci sono ore da assegnare
    while ore_totali > 0:
        miglior_punteggio = -float('inf')
        migliore_assegnazione = None

        # Per ogni combinazione (prof, classe, giorno, slot)
        for prof, assegnazioni in cattedre_rimanenti.items():
            for classe, ore in assegnazioni:
                if ore <= 0:
                    continue

                for giorno in giorni:
                    # Verifica giorno libero già assegnato
                    if prof in giorni_liberi_assegnati and giorni_liberi_assegnati[prof] == giorno:
                        continue

                    # Verifica limite ore giornaliere
                    if ore_prof_giorno[prof][giorno] >= MAX_ORE_GIORNO:
                        continue

                    for slot in slots:
                        # Verifica sovrapposizioni
                        if (giorno, slot, prof) in slot_assegnati or (giorno, slot, classe) in slot_assegnati:
                            continue

                        # Calcola punteggio completo con tutti i vincoli
                        punteggio = calcola_punteggio(prof, classe, giorno, slot, orario, cattedre_rimanenti, preferenze_giorni)

                        if punteggio > float('-inf') and punteggio > miglior_punteggio:
                            miglior_punteggio = punteggio
                            migliore_assegnazione = (prof, classe, giorno, slot)

        # Se non è stato trovato uno slot valido, interrompi
        if migliore_assegnazione is None:
            print(f"Non è possibile assegnare ulteriori ore. Rimangono {ore_totali} ore da assegnare.")
            break

        # Assegna la lezione
        prof, classe, giorno, slot = migliore_assegnazione

        orario.append((giorno, slot, prof, classe))
        slot_assegnati.add((giorno, slot, classe))
        slot_assegnati.add((giorno, slot, prof))

        # Aggiorna ore assegnate per giorno
        ore_prof_giorno[prof][giorno] += 1

        # Verifica se il prof non ha altri giorni disponibili e assegna giorno libero
        if prof not in giorni_liberi_assegnati:
            giorni_utilizzati = sum(1 for g, val in ore_prof_giorno[prof].items() if val > 0)
            if giorni_utilizzati >= len(giorni) - 1:
                giorni_disponibili = [g for g in giorni if ore_prof_giorno[prof][g] == 0]
                if giorni_disponibili:
                    # Scegli il giorno libero con priorità alta
                    giorni_disponibili.sort(key=lambda g: preferenze_giorni.get(prof, {}).get(g, 0), reverse=True)
                    giorni_liberi_assegnati[prof] = giorni_disponibili[0]

        # Aggiorna le ore rimanenti
        for i, (c, o) in enumerate(cattedre_rimanenti[prof]):
            if c == classe:
                cattedre_rimanenti[prof][i] = (c, o - 1)
                break

        ore_totali -= 1

    # Verifica se tutte le ore sono state assegnate
    ore_assegnate = len(orario)
    if ore_assegnate < ore_iniziali:
        print(f"ATTENZIONE: Assegnate solo {ore_assegnate} ore su {ore_iniziali}!")

        # Dettaglio delle ore non assegnate
        cattedre_rimanenti_dettaglio = {}
        for prof, assegnazioni in cattedre_rimanenti.items():
            ore_rimanenti = []
            for classe, ore in assegnazioni:
                if ore > 0:
                    ore_rimanenti.append((classe, ore))
            if ore_rimanenti:
                cattedre_rimanenti_dettaglio[prof] = ore_rimanenti

        for prof, assegnazioni in cattedre_rimanenti_dettaglio.items():
            for classe, ore in assegnazioni:
                print(f"  {prof}: {ore} ore rimanenti in {classe}")
    else:
        print(f"Tutte le {ore_iniziali} ore sono state assegnate con successo!")

    # Assegna giorni liberi ai professori che non ne hanno ancora
    for prof in cattedre.keys():
        if prof not in giorni_liberi_assegnati:
            # Calcola il giorno con meno ore o il giorno preferito
            giorni_ore = {giorno: sum(1 for g, _, p, _ in orario if p == prof and g == giorno) for giorno in giorni}
            giorni_candidati = [g for g, ore in giorni_ore.items() if ore == min(giorni_ore.values())]
            if giorni_candidati:
                giorni_candidati.sort(key=lambda g: preferenze_giorni.get(prof, {}).get(g, 0), reverse=True)
                giorni_liberi_assegnati[prof] = giorni_candidati[0]

    return orario

## 7) Funzione per visualizzare l'orario

Questa sezione implementa la funzionalità di visualizzazione dell'orario in formato tabellare. La funzione `visualizza_orario()` trasforma la rappresentazione interna dell'orario, costituita da una lista di tuple (giorno, slot, professore, classe), in una serie di DataFrame di pandas facilmente leggibili, uno per ciascuna classe. Ogni DataFrame ha come righe gli slot orari (da 1 a 5) e come colonne i giorni della settimana, con le celle popolate dal nome del professore che insegna in quel determinato momento. Questo formato matriciale rende immediata la consultazione dell'orario da parte di studenti e docenti, permettendo di visualizzare a colpo d'occhio la distribuzione settimanale delle lezioni per ogni classe. Il dizionario restituito `orari_per_classe` consente un accesso rapido all'orario di qualsiasi classe specifica.

In [7]:

# Funzione per visualizzare l'orario generato
def visualizza_orario(orario, classi, giorni, slots):
    # Crea un DataFrame vuoto per ogni classe
    orari_per_classe = {}
    for classe in classi:
        df = pd.DataFrame(index=slots, columns=giorni)
        df.index.name = "Slot"
        orari_per_classe[classe] = df

    # Riempi i DataFrame con le assegnazioni
    for giorno, slot, prof, classe in orario:
        orari_per_classe[classe].at[slot, giorno] = prof

    return orari_per_classe


## 8) Fase di ricerca locale

Questo capitolo implementa l'algoritmo di ricerca locale per ottimizzare l'orario generato dalla fase greedy iniziale. La funzione principale `ottimizza_orario_ricerca_locale()` esplora iterativamente lo spazio delle soluzioni applicando piccole modifiche all'orario corrente attraverso la funzione `genera_mossa()`, che implementa due tipi di operazioni: scambio di slot tra due professori e spostamento di una lezione a un diverso giorno/slot. Ogni mossa viene valutata controllando l'ammissibilità tramite `verifica_ammissibilita()`, che verifica il rispetto dei vincoli fondamentali come assenza di sovrapposizioni, limiti di ore giornaliere e sequenze di lezioni valide. La funzione `calcola_valore_soluzione()` quantifica la qualità dell'orario considerando molteplici fattori: soddisfazione delle preferenze sui giorni liberi (peso +50 per giorno preferito libero), penalità per ore buche (-20 per ciascuna), penalità per distribuzione non uniforme delle ore, e bonus per lezioni consecutive (+5). L'algoritmo accetta solo mosse migliorative, terminando quando non si trovano miglioramenti per un numero prestabilito di tentativi consecutivi (1000 per default) o si raggiunge il limite massimo di iterazioni (5000). Le funzioni ausiliarie `conta_ore_assegnate()`, `trova_slot_vuoti()`, `completa_orario()` e `verifica_vincoli()` supportano il processo garantendo la completezza dell'orario e il rispetto dei vincoli, anche attraverso il rilassamento progressivo di alcuni requisiti quando necessario per completare tutti gli slot.

In [8]:
def ottimizza_orario_ricerca_locale(orario_iniziale, cattedre, preferenze_giorni,
                                    max_iterazioni=5000, max_tentativi_senza_miglioramento=1000):
    """
    Ottimizza l'orario usando la ricerca locale attraverso mosse di scambio e spostamento.
    """
    orario_corrente = copy.deepcopy(orario_iniziale)
    valore_corrente = calcola_valore_soluzione(orario_corrente, preferenze_giorni)

    orario_migliore = copy.deepcopy(orario_corrente)
    valore_migliore = valore_corrente

    iterazioni_senza_miglioramento = 0

    print(f"Valore iniziale della soluzione: {valore_corrente}")

    for iterazione in range(max_iterazioni):
        # Genera una nuova soluzione attraverso una mossa
        nuovo_orario = genera_mossa(orario_corrente)

        # Verifica se la nuova soluzione è ammissibile
        if verifica_ammissibilita(nuovo_orario, cattedre):
            # Calcola il valore della nuova soluzione
            nuovo_valore = calcola_valore_soluzione(nuovo_orario, preferenze_giorni)

            # Accetta la mossa se migliora la soluzione
            if nuovo_valore > valore_corrente:
                orario_corrente = nuovo_orario
                valore_corrente = nuovo_valore
                iterazioni_senza_miglioramento = 0

                # Aggiorna la miglior soluzione trovata
                if valore_corrente > valore_migliore:
                    orario_migliore = copy.deepcopy(orario_corrente)
                    valore_migliore = valore_corrente
                    print(f"Nuova miglior soluzione trovata alla iterazione {iterazione} con valore {valore_migliore}")
            else:
                iterazioni_senza_miglioramento += 1
        else:
            iterazioni_senza_miglioramento += 1

        # Criteri di terminazione
        if iterazioni_senza_miglioramento >= max_tentativi_senza_miglioramento:
            print(f"Terminazione anticipata dopo {iterazione+1} iterazioni: nessun miglioramento per {max_tentativi_senza_miglioramento} tentativi")
            break

    print(f"Ricerca locale completata. Valore finale: {valore_migliore}")
    return orario_migliore


def verifica_ammissibilita(orario, cattedre):
    """
    Verifica se una soluzione è ammissibile (rispetta i vincoli fondamentali).
    Restituisce True se la soluzione è ammissibile, False altrimenti.
    """
    # Verifica sovrapposizioni
    slot_occupati = set()
    for giorno, slot, prof, classe in orario:
        # Verifica che non ci siano sovrapposizioni di prof o classe nello stesso slot
        if (giorno, slot, prof) in slot_occupati or (giorno, slot, classe) in slot_occupati:
            return False
        slot_occupati.add((giorno, slot, prof))
        slot_occupati.add((giorno, slot, classe))

    # Verifica ore massime giornaliere
    for prof in set(p for _, _, p, _ in orario):
        for giorno in GIORNI:
            ore_prof_giorno = sum(1 for g, _, p, _ in orario if p == prof and g == giorno)
            if ore_prof_giorno > MAX_ORE_GIORNO:
                return False
            """
            # Verifica massimo 2 lezioni dello stesso prof nella stessa classe per giorno
            for classe in set(c for _, _, p, c in orario if p == prof):
                lezioni_stessa_classe = sum(1 for g, _, p2, c in orario
                                           if g == giorno and p2 == prof and c == classe)
                if lezioni_stessa_classe > 2:
                    return False


            # Verifica ore buche (massimo 1 al giorno)
            slots_occupati = sorted([s for g, s, p, _ in orario if g == giorno and p == prof])
            ore_buche = ha_ore_buche(slots_occupati)
            if ore_buche > MAX_ORE_BUCHE:
                return False
            """
    # Verifica sequenza lezioni (non distanziate di 2+ slot)
    for prof in set(p for _, _, p, _ in orario):
        for classe in set(c for _, _, p2, c in orario if p2 == prof):
            for giorno in GIORNI:
                slots_classe_prof = sorted([s for g, s, p2, c2 in orario
                                          if g == giorno and p2 == prof and c2 == classe])
                if len(slots_classe_prof) >= 2:
                    for i in range(len(slots_classe_prof) - 1):
                        if slots_classe_prof[i+1] - slots_classe_prof[i] >= 3:  # distanza di 2+ slot
                            return False
    """
    # Verifica ore totali per ogni cattedra
    ore_assegnate, _ = conta_ore_assegnate(orario, cattedre)
    for prof, assegnazioni in cattedre.items():
        for classe, ore_richieste in assegnazioni:
            ore_effettive = ore_assegnate[prof][classe]
            if ore_effettive > ore_richieste:  # Possiamo avere meno ore, ma non più
                return False
    """

    return True
def genera_mossa(orario):
    """
    Genera una mossa casuale per la ricerca locale:
    - Scambio di slot tra due professori
    - Spostamento di una lezione a un altro giorno/slot
    """
    tipo_mossa = random.choice(["scambio", "spostamento"])

    if tipo_mossa == "scambio" and len(orario) >= 2:
        # Scegliamo due lezioni da scambiare
        idx1, idx2 = random.sample(range(len(orario)), 2)
        giorno1, slot1, prof1, classe1 = orario[idx1]
        giorno2, slot2, prof2, classe2 = orario[idx2]

        # Creiamo una copia dell'orario
        nuovo_orario = copy.deepcopy(orario)
        # Effettuiamo lo scambio
        nuovo_orario[idx1] = (giorno2, slot2, prof1, classe1)
        nuovo_orario[idx2] = (giorno1, slot1, prof2, classe2)

        return nuovo_orario
    else:
        # Spostamento di una lezione
        idx = random.randrange(len(orario))
        giorno_attuale, slot_attuale, prof, classe = orario[idx]

        # Scegli un nuovo giorno e slot
        nuovo_giorno = random.choice(GIORNI)
        nuovo_slot = random.choice(SLOTS)

        # Se è lo stesso slot, non fare nulla
        if nuovo_giorno == giorno_attuale and nuovo_slot == slot_attuale:
            return orario

        # Creiamo una copia dell'orario e aggiorniamo
        nuovo_orario = copy.deepcopy(orario)
        nuovo_orario[idx] = (nuovo_giorno, nuovo_slot, prof, classe)

        return nuovo_orario
def calcola_valore_soluzione(orario, preferenze_giorni):
    """
    Calcola il valore della soluzione in base a diversi fattori:
    - Preferenze dei giorni liberi soddisfatte
    - Numero di ore buche
    - Distribuzione delle ore
    """
    punteggio = 0

    # Calcola i giorni liberi per ogni professore
    giorni_prof = defaultdict(set)
    for giorno, _, prof, _ in orario:
        giorni_prof[prof].add(giorno)

    # Giorni liberi e soddisfazione preferenze
    for prof, giorni_occupati in giorni_prof.items():
        giorni_liberi = set(GIORNI) - giorni_occupati
        for giorno in giorni_liberi:
            punteggio += preferenze_giorni.get(prof, {}).get(giorno, 0) * 50

    # Penalizza ore buche
    for prof in set(p for _, _, p, _ in orario):
        for giorno in GIORNI:
            slots_occupati = sorted([s for g, s, p, _ in orario if g == giorno and p == prof])
            ore_buche = 0
            for i in range(len(slots_occupati) - 1):
                gap = slots_occupati[i + 1] - slots_occupati[i] - 1
                ore_buche += max(0, gap)
            punteggio -= ore_buche * 20  # Penalità per ogni ora buca

    # Premia distribuzione equa delle ore
    for prof in set(p for _, _, p, _ in orario):
        ore_per_giorno = Counter(g for g, _, p, _ in orario if p == prof)
        std_dev = np.std(list(ore_per_giorno.values())) if ore_per_giorno else 0
        punteggio -= std_dev * 10  # Penalità per distribuzione non uniforme

    # Premia continuità delle lezioni (slot consecutivi)
    for prof in set(p for _, _, p, _ in orario):
        for giorno in GIORNI:
            slots_prof = sorted([s for g, s, p, _ in orario if g == giorno and p == prof])
            for i in range(len(slots_prof) - 1):
                if slots_prof[i + 1] == slots_prof[i] + 1:  # Slot consecutivi
                    punteggio += 5  # Bonus per continuità

    return punteggio

def conta_ore_assegnate(orario, cattedre):
    """
    Conta le ore assegnate per ogni professore e classe
    e confronta con il numero richiesto.
    """
    ore_assegnate = defaultdict(lambda: defaultdict(int))
    for _, _, prof, classe in orario:
        ore_assegnate[prof][classe] += 1

    ore_mancanti = {}
    for prof, assegnazioni in cattedre.items():
        prof_ore_mancanti = {}
        for classe, ore_richieste in assegnazioni:
            assegnate = ore_assegnate[prof][classe]
            mancanti = ore_richieste - assegnate
            if mancanti > 0:
                prof_ore_mancanti[classe] = mancanti
        if prof_ore_mancanti:
            ore_mancanti[prof] = prof_ore_mancanti

    return ore_assegnate, ore_mancanti

## 9) Main

Questa sezione contiene la funzione principale `main()` che orchestra l'intero processo di generazione e ottimizzazione dell'orario scolastico attraverso tre fasi sequenziali. La prima fase utilizza l'algoritmo greedy per creare un orario iniziale attraverso `orario_scolastico()`, assegnando le lezioni in modo incrementale e intelligente. La seconda fase completa eventuali slot rimasti vuoti tramite `completa_orario()`, che tenta di riempire tutti gli spazi utilizzando un approccio a vincoli progressivamente rilassati per garantire la copertura totale dell'orario. La terza fase applica la ricerca locale tramite `ottimizza_orario_ricerca_locale()` per migliorare la qualità complessiva della soluzione, raffinando la distribuzione delle lezioni e massimizzando la soddisfazione dei vincoli soft. Al termine dell'ottimizzazione, la funzione assegna i giorni liberi a ciascun professore attraverso `assegna_giorni_liberi()`, privilegiando le preferenze individuali quando possibile. Infine, presenta una valutazione dettagliata della soluzione finale includendo il punteggio complessivo, il numero di slot vuoti rimasti, e statistiche approfondite sulla soddisfazione delle preferenze per i giorni liberi, distinguendo tra prima scelta, seconda scelta e altre opzioni, con calcolo delle percentuali per fornire una chiara visione della qualità dell'orario generato.

In [9]:
def main():
    """
    Funzione principale che esegue l'algoritmo greedy iniziale seguito da
    completamento e ottimizzazione dell'orario tramite ricerca locale.
    """

    print("FASE 1: Creazione orario iniziale...")
    orario = orario_scolastico(cattedre, CLASSI, GIORNI, SLOTS, preferenze_giorni)

    # Calcola le cattedre rimanenti
    _, ore_mancanti = conta_ore_assegnate(orario, cattedre)

    print("\nFASE 2: Completamento dell'orario")
    orario_completo = completa_orario(orario, cattedre, CLASSI, GIORNI, SLOTS)

    print("\nFASE 3: Ottimizzazione dell'orario tramite ricerca locale...")
    orario_ottimizzato = ottimizza_orario_ricerca_locale(orario_completo, cattedre, preferenze_giorni)

    # Assegna giorni liberi in base all'orario ottimizzato
    giorni_liberi_ottimizzati = assegna_giorni_liberi(orario_ottimizzato, preferenze_giorni)

    # Visualizza l'orario ottimizzato
    orari_per_classe = visualizza_orario(orario_ottimizzato, CLASSI, GIORNI, SLOTS)

    # Valutazione finale
    print("\nValutazione finale dell'orario ottimizzato:")
    valore_soluzione = calcola_valore_soluzione(orario_ottimizzato, preferenze_giorni)
    print(f"Punteggio della soluzione: {valore_soluzione}")

    slot_vuoti_rimasti = trova_slot_vuoti(orario_ottimizzato, CLASSI, GIORNI, SLOTS)
    print(f"Slot vuoti rimasti: {len(slot_vuoti_rimasti)}")

    # Statistiche sui giorni liberi
    print("\nGiorni liberi assegnati:")
    for prof, giorno in sorted(giorni_liberi_ottimizzati.items()):
        preferenza = preferenze_giorni[prof][giorno] if giorno else "N/A"
        if preferenza == 2:
            soddisfazione = "Prima scelta"
        elif preferenza == 1:
            soddisfazione = "Seconda scelta"
        else:
            soddisfazione = "Altra scelta"

        print(f"{prof}: {giorno if giorno else 'Nessun giorno libero'} ({soddisfazione})")

    print("\nStatistiche sulla soddisfazione delle preferenze:")
    prima_scelta = sum(1 for prof, giorno in giorni_liberi_ottimizzati.items()
                      if giorno and preferenze_giorni[prof][giorno] == 2)
    seconda_scelta = sum(1 for prof, giorno in giorni_liberi_ottimizzati.items()
                       if giorno and preferenze_giorni[prof][giorno] == 1)
    altra_scelta = sum(1 for prof, giorno in giorni_liberi_ottimizzati.items()) - prima_scelta - seconda_scelta


    n_professori = len(cattedre)
    print(f"Prima scelta: {prima_scelta}/{n_professori} ({prima_scelta/n_professori*100:.1f}%)")
    print(f"Seconda scelta: {seconda_scelta}/{n_professori} ({seconda_scelta/n_professori*100:.1f}%)")
    print(f"Altra scelta: {altra_scelta}/{n_professori} ({altra_scelta/n_professori*100:.1f}%)")

    return orario_ottimizzato, giorni_liberi_ottimizzati, orari_per_classe

def assegna_giorni_liberi(orario, preferenze_giorni):
    """
    Assegna giorni liberi in base all'orario generato e alle preferenze.
    """
    giorni_liberi = {}

    for prof in set(p for _, _, p, _ in orario):
        # Trova i giorni in cui il professore ha lezioni
        giorni_con_lezioni = set(giorno for giorno, _, p, _ in orario if p == prof)

        # Giorni disponibili come liberi
        giorni_disponibili = [g for g in GIORNI if g not in giorni_con_lezioni]

        if giorni_disponibili:
            # Ordina in base alle preferenze (più alto è meglio)
            giorni_disponibili.sort(key=lambda g: preferenze_giorni.get(prof, {}).get(g, 0), reverse=True)
            giorni_liberi[prof] = giorni_disponibili[0]
        else:
            # Nel caso tutti i giorni abbiano lezioni (situazione non ottimale)
            giorni_liberi[prof] = None

    return giorni_liberi

def completa_orario(orario, cattedre, classi, giorni, slots):
    """
    Completa l'orario riempiendo tutti gli slot vuoti rispettando il più possibile i vincoli.
    Se necessario, rilassa alcuni vincoli per garantire la completezza dell'orario.
    """
    orario_completo = copy.deepcopy(orario)

    # Identifica le ore mancanti per ogni professore
    _, ore_mancanti = conta_ore_assegnate(orario_completo, cattedre)

    # Identifica gli slot vuoti da riempire
    slot_vuoti = trova_slot_vuoti(orario_completo, classi, giorni, slots)

    if not slot_vuoti:
        print("L'orario è già completo!")
        return orario_completo

    print(f"Completamento orario: {len(slot_vuoti)} slot vuoti da riempire")

    # Prima fase: tentativo con vincoli rigorosi
    for giorno, slot, classe in list(slot_vuoti):
        # Troviamo professori che devono ancora insegnare in questa classe
        for prof, classi_mancanti in ore_mancanti.items():
            if classe in classi_mancanti and classi_mancanti[classe] > 0:
                # Verifica se l'assegnazione rispetta i vincoli
                if verifica_vincoli(prof, classe, giorno, slot, orario_completo):
                    # Assegna la lezione
                    orario_completo.append((giorno, slot, prof, classe))
                    # Aggiorna le ore mancanti
                    ore_mancanti[prof][classe] -= 1
                    if ore_mancanti[prof][classe] == 0:
                        del ore_mancanti[prof][classe]
                        if not ore_mancanti[prof]:
                            del ore_mancanti[prof]
                    slot_vuoti.remove((giorno, slot, classe))
                    break

    # Seconda fase: rilassiamo alcuni vincoli per gli slot ancora vuoti
    for giorno, slot, classe in list(slot_vuoti):
        for prof, classi_mancanti in list(ore_mancanti.items()):
            if classe in classi_mancanti and classi_mancanti[classe] > 0:
                # Verifica con vincoli rilassati
                if verifica_vincoli(prof, classe, giorno, slot, orario_completo, strict=False):
                    orario_completo.append((giorno, slot, prof, classe))
                    ore_mancanti[prof][classe] -= 1
                    if ore_mancanti[prof][classe] == 0:
                        del ore_mancanti[prof][classe]
                        if not ore_mancanti[prof]:
                            del ore_mancanti[prof]
                    slot_vuoti.remove((giorno, slot, classe))
                    break

    # Terza fase: se ci sono ancora slot vuoti, assegniamo professori che hanno ore disponibili
    for giorno, slot, classe in list(slot_vuoti):
        for prof in cattedre.keys():
            # Verifica se il prof può insegnare in questa classe (basato sulle cattedre originali)
            if any(c == classe for c, _ in cattedre[prof]):
                if verifica_vincoli(prof, classe, giorno, slot, orario_completo, strict=False):
                    orario_completo.append((giorno, slot, prof, classe))
                    slot_vuoti.remove((giorno, slot, classe))
                    break

    print(f"Slot vuoti rimasti dopo il completamento: {len(slot_vuoti)}")
    return orario_completo

def verifica_vincoli(prof, classe, giorno, slot, orario_attuale, strict=True):
    """
    Verifica se l'assegnazione di un professore a uno slot rispetta i vincoli.
    Restituisce True se è possibile assegnare, False altrimenti.
    """
    # Verifica sovrapposizioni (stesso slot, stesso giorno per prof o classe)
    for g, s, p, c in orario_attuale:
        if g == giorno and s == slot:
            if p == prof or c == classe:
                return False  # Sovrapposizione trovata

    # Verifica ore massime giornaliere
    ore_prof_giorno = sum(1 for g, _, p, _ in orario_attuale if p == prof and g == giorno)
    if ore_prof_giorno >= MAX_ORE_GIORNO:
        return False  # Superato il limite di ore giornaliere

    # Calcola ore buche
    slots_occupati = sorted([s for g, s, p, _ in orario_attuale if g == giorno and p == prof] + [slot])
    ore_buche = 0
    for i in range(len(slots_occupati) - 1):
        gap = slots_occupati[i + 1] - slots_occupati[i] - 1
        ore_buche += max(0, gap)

    if ore_buche > MAX_ORE_BUCHE and strict:
        return False  # Troppe ore buche

    # Verifica lezioni dello stesso prof nella stessa classe nello stesso giorno
    lezioni_stessa_classe = sum(1 for g, _, p, c in orario_attuale
                               if g == giorno and p == prof and c == classe)
    if lezioni_stessa_classe >= 2 and strict:  # Massimo 2 lezioni per classe al giorno
        return False

    return True




def trova_slot_vuoti(orario, classi, giorni, slots):
    """
    Identifica gli slot orari che non sono stati assegnati.
    """
    slot_vuoti = []
    for classe in classi:
        for giorno in giorni:
            for slot in slots:
                if not any(g == giorno and s == slot and c == classe for g, s, _, c in orario):
                    slot_vuoti.append((giorno, slot, classe))
    return slot_vuoti

## 10) Analisi dei risultati

Questo blocco implementa la funzione `analizza_risultati()` che effettua un'analisi statistica completa dell'orario generato. La funzione raccoglie e organizza diverse metriche chiave: il numero totale di professori coinvolti e di assegnazioni effettuate, la distribuzione delle ore per ciascun professore nei vari giorni della settimana attraverso un dizionario nidificato, e un conteggio dei giorni liberi per identificare quali giorni della settimana sono più frequentemente assegnati come liberi. Inoltre, calcola il livello di soddisfazione delle preferenze confrontando, per ogni professore, il peso del giorno libero assegnato con il massimo peso tra le sue preferenze. Questa analisi quantitativa permette di valutare oggettivamente la qualità della soluzione ottenuta, evidenziando eventuali sbilanciamenti nella distribuzione del carico di lavoro e fornendo informazioni utili per successive iterazioni o miglioramenti dell'algoritmo. I dati restituiti in formato dizionario strutturato facilitano ulteriori elaborazioni e visualizzazioni.

In [10]:
# Analisi dei risultati
def analizza_risultati(orario, giorni_liberi, cattedre, preferenze_giorni):
    # Statistiche sull'orario generato
    professori = list(cattedre.keys())
    n_professori = len(professori)
    n_assegnazioni = len(orario)

    # Ore per giorno per ogni professore
    ore_per_giorno_prof = defaultdict(lambda: defaultdict(int))
    for giorno, slot, prof, classe in orario:
        ore_per_giorno_prof[prof][giorno] += 1

    # Statistiche sui giorni liberi
    giorni_liberi_count = Counter(giorni_liberi.values())

    # Statistiche sul soddisfacimento delle preferenze
    soddisfazione_preferenze = {}
    for prof, giorno in giorni_liberi.items():
        if giorno:
            # Punteggio di preferenza (più alto è meglio)
            preferenza = preferenze_giorni[prof][giorno]
            soddisfazione_preferenze[prof] = (preferenza, max(preferenze_giorni[prof].values()))

    return {
        "n_professori": n_professori,
        "n_assegnazioni": n_assegnazioni,
        "ore_per_giorno_prof": dict(ore_per_giorno_prof),
        "giorni_liberi": giorni_liberi,
        "giorni_liberi_count": dict(giorni_liberi_count),
        "soddisfazione_preferenze": soddisfazione_preferenze
    }


## 11) Esecuzione algoritmo e stampa delle statistiche

In questa sezione finale viene eseguito l'intero processo di generazione dell'orario e vengono visualizzati i risultati ottenuti. Inizialmente vengono elaborati i dati delle cattedre tramite `leggi_cattedre()` e generate le preferenze per i giorni liberi con `genera_preferenze_giorni_liberi()`. Successivamente viene invocata la funzione `main()` che coordina tutte le fasi di costruzione e ottimizzazione dell'orario, restituendo la soluzione finale, i giorni liberi assegnati e gli orari formattati per classe. La funzione `analizza_risultati()` viene quindi chiamata per produrre le statistiche dettagliate sulla soluzione. Infine, un ciclo itera su tutte le classi stampando a video l'orario completo di ciascuna in formato tabellare, permettendo una verifica immediata e intuitiva del risultato. Questo output finale mostra chiaramente la distribuzione settimanale delle lezioni per ogni classe, con i nomi dei professori assegnati a ciascuno slot, fornendo una rappresentazione completa e facilmente consultabile dell'orario scolastico generato dall'algoritmo.

In [11]:

# Esegui l'algoritmo e analizza i risultati
# Elabora i dati delle cattedre
cattedre = leggi_cattedre(dati_cattedre)
# Genera preferenze per i giorni liberi
preferenze_giorni = genera_preferenze_giorni_liberi(cattedre.keys(), GIORNI)
orario, giorni_liberi, orari_per_classe = main()  # Assign to 3 variables
analisi = analizza_risultati(orario, giorni_liberi, leggi_cattedre(dati_cattedre), preferenze_giorni)  # preferenze_giorni is already defined

# Visualizza l'orario di tutte le classi
for classe in CLASSI:
    print(f"\nOrario della classe {classe}:")
    print(orari_per_classe[classe])

FASE 1: Creazione orario iniziale...
Ore totali da assegnare: 180
Non è possibile assegnare ulteriori ore. Rimangono 9 ore da assegnare.
ATTENZIONE: Assegnate solo 171 ore su 180!
  Young: 2 ore rimanenti in 4A
  Green: 1 ore rimanenti in 5C
  Gonzalez: 1 ore rimanenti in 5C
  Carter: 1 ore rimanenti in 4C
  Mitchell: 3 ore rimanenti in 3A
  Mitchell: 1 ore rimanenti in 5B

FASE 2: Completamento dell'orario
Completamento orario: 9 slot vuoti da riempire
Slot vuoti rimasti dopo il completamento: 0

FASE 3: Ottimizzazione dell'orario tramite ricerca locale...
Valore iniziale della soluzione: 4393.406084430867
Terminazione anticipata dopo 1000 iterazioni: nessun miglioramento per 1000 tentativi
Ricerca locale completata. Valore finale: 4393.406084430867

Valutazione finale dell'orario ottimizzato:
Punteggio della soluzione: 4393.406084430867
Slot vuoti rimasti: 0

Giorni liberi assegnati:
Adams: Mercoledì (Prima scelta)
Allen: Lunedì (Prima scelta)
Anderson: Venerdì (Prima scelta)
Baker: 